In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [4]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [5]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.52285683
y: 0.537446
z: -1.7137096
visibility: 0.99448353
, x: 0.574737
y: 0.44992828
z: -1.6630633
visibility: 0.986485
, x: 0.6014658
y: 0.45092514
z: -1.6633646
visibility: 0.98782104
, x: 0.6253226
y: 0.45261896
z: -1.6639478
visibility: 0.9851393
, x: 0.503574
y: 0.44217157
z: -1.6450012
visibility: 0.9882677
, x: 0.48283672
y: 0.43734467
z: -1.6439499
visibility: 0.9901653
, x: 0.46584576
y: 0.43411005
z: -1.6448877
visibility: 0.9893267
, x: 0.6635327
y: 0.47697476
z: -1.0624808
visibility: 0.988555
, x: 0.4575306
y: 0.44318736
z: -0.9401866
visibility: 0.9937762
, x: 0.57006794
y: 0.6165687
z: -1.4682995
visibility: 0.99630165
, x: 0.47603613
y: 0.6067302
z: -1.4344938
visibility: 0.996588
, x: 0.8610102
y: 0.9154376
z: -0.521674
visibility: 0.99534756
, x: 0.28538665
y: 0.7271486
z: -0.54331124
visibility: 0.9986791
, x: 0.94650924
y: 1.4037684
z: -0.27158892
visibility: 0.14483275
, x: 0.005362481
y: 1.0376027
z: -0.45662612
visibility: 0.72659427
, x: 0.9057683
y: 1.84

In [6]:
len(landmarks)

33

In [7]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [8]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.29099333
y: 1.716937
z: 0.026814798
visibility: 0.00053853873

In [9]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

x: 0.2598581
y: 2.5582325
z: 0.062389188
visibility: 0.00014125672

In [10]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.26936513
y: 0.7251465
z: -0.82498753
visibility: 0.99972975

In [11]:
def calculate_deadlift_angle(right_shoulder, right_hip, right_knee):
    right_shoulder = np.array(right_shoulder)  
    right_hip = np.array(right_hip)  
    right_knee = np.array(right_knee)  
    
    # Calculate the angle between hip, knee, and shoulder
    radians = np.arctan2(right_knee[1] - right_hip[1], right_knee[0] - right_hip[0]) - np.arctan2(right_shoulder[1] - right_hip[1], right_shoulder[0] - right_hip[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle


In [12]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_hip= [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
right_knee= [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

In [13]:
right_shoulder, right_hip, right_knee

([0.269365131855011, 0.725146472454071],
 [0.290993332862854, 1.7169369459152222],
 [0.259858101606369, 2.558232545852661])

In [14]:
calculate_deadlift_angle(right_shoulder, right_hip, right_knee)

176.6312625658471

In [15]:
# Convert landmark coordinates to pixel values
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_hip, [640, 480]).astype(int))
tuple(np.multiply(right_knee, [640, 480]).astype(int))


(166, 1227)

Start of deadlift form logic 

In [72]:
cap = cv2.VideoCapture(0)

# Deadlift counter variables
counter = 0
stage = None
feedback = "Start the lift"

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for deadlift landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate deadlift angle
            angle = calculate_deadlift_angle(right_shoulder, right_hip, right_knee)  # Modify function for deadlift angle
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Deadlift counter logic
            if angle > 170:
                stage = "up"
            if angle < 90 and stage == 'up':
                stage = "down"
                counter += 1
                print(counter)
                       
        except:
            pass
        
        # Render deadlift counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'DEADLIFTS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Deadlift Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [73]:
cap = cv2.VideoCapture(0)

# Deadlift counter variables
counter = 0
stage = None
feedback = "Start the lift"

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            
            # Get coordinates for deadlift landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate deadlift angle
            angle = calculate_deadlift_angle(right_shoulder, right_hip, right_knee)  # Modify function for deadlift angle
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Deadlift counter logic
            # You need to define the specific logic for deadlift completion
            # For example, you might check for when the angle exceeds a certain threshold
            
            if angle > 170:
                stage = "up"
            if angle < 90 and stage == 'up':
                stage = "down"
                counter += 1

            if angle < 45 and stage == "down":
                feedback = "Get more upright"
            if angle > 80 and stage =="down":
                feedback = "Get more bent over"
                
                       
        except:
            pass
        
        # Render deadlift counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        cv2.putText(image, 'DEADLIFTS', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'STAGE'
        cv2.putText(image, 'STAGE', (165, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (160, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'FORM' (Ensure it fits within the box)
        feedback_lines = feedback.split('\n')
        if len(feedback_lines) > 0:
            feedback_line1 = feedback_lines[0]
        else:
            feedback_line1 = ''

        if len(feedback_lines) > 1:
            feedback_line2 = feedback_lines[1]
        else:
            feedback_line2 = ''

        cv2.putText(image, 'FORM', (290, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, feedback_line1, (270, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, feedback_line2, (270, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        
        cv2.imshow('Deadlift Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
